In [ ]:
!pip install --upgrade pip

#tweepy library 
!pip install tweepy
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler

#socket for connection with Twitter
import socket

#json for tweet loading
import json

In [ ]:
#TWITTER API v2 (v1 is deprecated)

class TweetsListener(tweepy.StreamingClient):
    
    def __init__(self, *args, csocket):
        
        super().__init__(*args)
        self.client_socket = csocket
    
    
    def on_data(self, data):
        
        # load of tweets in json file
        try:  
            msg = json.loads(data)
            print("New message detected: ")
            self.client_socket\
                 .send(str(msg['data']['text']+"t_end")\
                 .encode('utf-8'))
            print(msg['data']['text'])
            return True
        except BaseException as e:
            print("Error: %s" %str(e))
        return True
    
    # visualize streaming errors (e.g. error 420 of disconnection)
    def on_error(self, status_code):
        
        print(status)
        return True

In [ ]:
# key and token for API v1 (deprecated)
consumer_key="<Insert your consumer_key>"
consumer_secret="<Insert your consumer_secret>"
access_token ="<Insert your access_token>"
access_secret="<Insert your access_secret>"

# token for API v2
BEARER_TOKEN = "<Insert your BEARER_TOKEN>"

In [ ]:
# Delete previous query to include latest tweets

twitter_stream = tweepy.StreamingClient(BEARER_TOKEN)

rule_ids = []
result = twitter_stream.get_rules()
try:
    for rule in result.data:
        print(f"rule marked to delete: {rule.id} - {rule.value}")
        rule_ids.append(rule.id)
    if (len(rule_ids) > 0):
        twitter_stream.delete_rules(rule_ids)
        twitter_stream = tweepy.StreamingClient(BEARER_TOKEN)
    else:
        print("no rules to delete")
except BaseException as e:
    print(f"Error on_data: {e}")

Error on_data: 'NoneType' object is not iterable


In [ ]:
# function to link Twitter and socket to manage data 

def send_data(socket):
    print('Sending data from Twitter to socket...')
    twitter_stream = TweetsListener(BEARER_TOKEN, csocket=socket)
    twitter_stream.add_rules(tweepy.StreamRule('("Dwayne Johnson" OR "Dwayne TheRock Johnson" OR @TheRock) -is:retweet lang:en'))
    twitter_stream.filter()

In [ ]:
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
host = "localhost"    
port = 3360
s.bind((host, port))
print("Socket is ready!")

s.listen(1)
print("Socket is listening...")

# return the socket and the address on the other side of the connection (client side)
c_socket, addr = s.accept()
print("Received request from: " + str(addr))

send_data(c_socket)